In [290]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")

D=6;
chi=10;
W=10;
N=20;
tol=1e-6;

Dtrun=W*W*chi*chi;

pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [243]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");


"initialize CTM"

"start CTM iterations:"

  0.249487 seconds (587.61 k allocations: 64.222 MiB, 43.67% gc time)
CTMRG iteration: 1, CTMRG err: 2.381843388312667
CTMRG iteration: 2, CTMRG err: 0.16758887210702916
CTMRG iteration: 3, CTMRG err: 0.01596027742626989
CTMRG iteration: 4, CTMRG err: 0.005665817296959171
CTMRG iteration: 5, CTMRG err: 0.0016753918020690128
CTMRG iteration: 6, CTMRG err: 0.0006074576485720598
CTMRG iteration: 7, CTMRG err: 0.00023375102816346817
CTMRG iteration: 8, CTMRG err: 9.659154953051407e-5
CTMRG iteration: 9, CTMRG err: 4.080785607776493e-5
CTMRG iteration: 10, CTMRG err: 1.7562204063620438e-5
CTMRG iteration: 11, CTMRG err: 7.617736490288161e-6
CTMRG iteration: 12, CTMRG err: 3.3195128247064078e-6
CTMRG iteration: 13, CTMRG err: 1.4495809776378003e-6
CTMRG iteration: 14, CTMRG err: 6.336851062992582e-7
  4.252772 seconds (10.70 M allocations: 1014.537 MiB, 12.02% gc time, 22.44% compilation time)
  2.513634 seconds (4.92 M allocations: 547.358 MiB, 30.51% gc time, 36.86% compilation time)
  1.7

In [244]:
display((E_up+E_down)/3)
Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];

@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];


-0.4272664643496391 + 6.7966121702560844e-18im

In [245]:
function HV_L_tensor(vl,A,mpo)
    if mpo==[]
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A'[3,-2,2]*A[1,-3,2];
        end
    else
        if numind(vl)==5
            @tensor vl[:]:=vl[-1,7,5,3,1]*A'[7,-2,6]*mpo'[5,6,-3,4]*mpo[3,2,-4,4]*A[1,-5,2];
        elseif numind(vl)==4
            @tensor vl[:]:=vl[7,5,3,1]*A'[7,-1,6]*mpo'[5,6,-2,4]*mpo[3,2,-3,4]*A[1,-4,2];
        end
    end
    return vl
end

function HV_R_tensor(vr,A,mpo)
    if mpo==[]
        if numind(vr)==3
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
        elseif numind(vr)==2
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3];
        end
    else
        if numind(vr)==5
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1,-5];
        elseif numind(vr)==4
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1];
        end
    end
    return vr
end

function left_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_L_tensor(x,A,[]);
        vl_init = permute(TensorMap(randn, space(A,1), space(A,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_L_tensor(x,A,mpo);
        vl_init = permute(TensorMap(randn, space(A,1)*space(mpo,1),space(mpo,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end

function right_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_R_tensor(x,A,[]);
        vr_init = permute(TensorMap(randn, space(A,2), space(A,2)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_R_tensor(x,A,mpo);
        vr_init = permute(TensorMap(randn, space(A,2)*space(mpo,3),space(mpo,3)*space(A,2)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end



function left_eigenvalue(A1,A2,n)
    function HV_L_A1A2(vl,A1,A2)
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A1'[3,-2,2]*A2[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A1'[3,-2,2]*A2[1,-3,2];
        end
    return vl
    end
    HVA1A2fun(x)=HV_L_A1A2(x,A1,A2);
    vl_init = permute(TensorMap(randn, space(A1,1), space(A2,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
    eu,ev=eigsolve(HVA1A2fun, vl_init, 1,:LM,Arnoldi());
    @assert maximum(abs.(eu)) == abs(eu[1])
    eu=eu[1:n];
    return eu
end

left_eigenvalue (generic function with 1 method)

In [246]:
function impo_imps(mpo,A)
    unitary_mpo_A=unitary(fuse(space(mpo,1)⊗space(A,1)), space(mpo,1)⊗space(A,1));
    @tensor mpo_A[:]:=unitary_mpo_A[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_mpo_A'[4,5,-2];
    return mpo_A
end

impo_imps (generic function with 1 method)

In [247]:
function itebd_ite(mpo,A,W,type)
    if type=="A"
        mpo_A=impo_imps(mpo,A);
        vl,_=left_eigenvector(mpo_A,[],"A");
        eu,ev=eig(permute(vl,(1,),(2,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,1]*mpo_A[1,2,-3]*unitary_R[2,-2];
    elseif type=="mpo_A"
        vl,_=left_eigenvector(A,mpo,"mpo_A");
        eu,ev=eig(permute(vl,(2,1,),(3,4,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_R[4,5,-2];
    end

    vr,_=right_eigenvector(AL,[],"A");

    U,S,_=tsvd(permute(vr,(1,),(2,)),trunc=truncdim(W));
    #display(diag(convert(Array,S)))
    @tensor A_trun[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];

    return A_trun
end


itebd_ite (generic function with 1 method)

In [248]:
function left_right_normalize(A)
    #left normalize
    vl,_=left_eigenvector(A,[],"A");
    eu,ev=eig(permute(vl,(1,),(2,)));
    unitary_L=sqrt(eu)*ev';
    unitary_R=ev*pinv(sqrt(eu));
    @tensor AL[:]:=unitary_L[-1,1]*A[1,2,-3]*unitary_R[2,-2];
    #transform right fixed point to be diagonal
    vr,eu=right_eigenvector(AL,[],"A");
    U,S,_=tsvd(permute(vr,(1,),(2,)));
    @tensor AL[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];
    AL=AL/sqrt(eu);
    return AL
end


left_right_normalize (generic function with 1 method)

In [250]:
function ITEBD_boundary_groundstate(O1,O2,W,A_init,method)
# ITEBD_boundary_groundstate(O1,O2,D,inv_tol):
    #use power method to obtain dominant boundary imps of nonhermitian MPO
    if method=="OO"
        @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
        U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
        @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    end
    #Initial state
    if A_init==[]
        mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
        A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());
    else 
    end
    AL=left_right_normalize(A_init);
    
    for cp=1:30
        AL_old=AL;
        @time if method=="O_O"
            A_trun_intermed=itebd_ite(O1,AL,3*W,"mpo_A");
            A_trun=itebd_ite(O2,A_trun_intermed,W,"mpo_A");
            AL=left_right_normalize(A_trun);
        elseif method=="OO"
            A_trun=itebd_ite(OO,AL,W,"mpo_A");#this "mpo_A" option is slightly faster when bond dimension is large
        end
        AL=left_right_normalize(A_trun);

        # vv,ee=right_eigenvector(AL,[],"A");
        # display(ee)
        # u,s,v=tsvd(permute(vv,(1,),(2,)));
        # display(sort(diag(convert(Array,s)),rev=true))
        
        #dominant eigenvalue of transfer matrix
        E=left_eigenvalue(impo_imps(O2,impo_imps(O1,AL)),AL,1)[1]
        ov=abs(left_eigenvalue(AL,AL_old,1)[1]);
        println("E="*string(E)*", "*"ov="*string(ov))
        if abs(ov-1)<1e-8
            break
        end
    
    end
    return AL,A_init
end

ITEBD_boundary_groundstate (generic function with 1 method)

In [291]:
try
    #load data
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    Ag_dict=load(jld_filenm)["Ag_dict"];
    O1_dict=load(jld_filenm)["O1_dict"];
    O2_dict=load(jld_filenm)["O2_dict"];
    Ag=convert(TensorMap,Ag_dict);
    O1=convert(TensorMap,O1_dict);
    O2=convert(TensorMap,O2_dict);
    display("load itebd ground state from saved data directly")
catch e

    display("No itebd ground state found from saved data, now calculate it")
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
    A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

    #Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
    Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

    #save itebd data
    O1_dict=convert(Dict,O1);
    O2_dict=convert(Dict,O2);
    Ag_dict=convert(Dict,Ag);
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

    #save initial CTM to compare with other codes
    matwrite("itebd_matfile.mat", Dict(
        "O1" => convert(Array,O1),
        "O2" => convert(Array,O2),
        "A_init" => convert(Array,A_init),
        "Ag" => convert(Array,Ag)
    ); compress = false)


end

"load itebd ground state from saved data directly"

In [292]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));

AOA_sec=collect(sectors(space_AOA))
AA_sec=collect(sectors(space_AA))

@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))

(Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [255]:
# W=20;chi=20;
# (Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1)' ⊗ Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
# Rep[SU₂](0=>4999, 1/2=>8314, 1=>9146, 3/2=>7850, 2=>5503, 5/2=>3190, 3=>1521, 7/2=>584, 4=>173, 9/2=>36, 5=>4)

In [256]:
for s in sectors(space_AOA)
  display(s)
  @show s, dim(space_AOA, s), dim(s)
end


Irrep[SU₂](0)

Irrep[SU₂](1/2)

Irrep[SU₂](1)

Irrep[SU₂](3/2)

Irrep[SU₂](2)

Irrep[SU₂](5/2)

Irrep[SU₂](3)

Irrep[SU₂](7/2)

Irrep[SU₂](4)

(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](0), 323, 1)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1/2), 512, 2)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1), 518, 3)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3/2), 392, 4)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](2), 230, 5)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](5/2), 104, 6)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3), 35, 7)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](7/2), 8, 8)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](4), 1, 9)


In [258]:

function group_numbers(E_set)
    Es=copy(E_set[1]);
    for cc=2:length(E_set)
        Es=vcat(Es,E_set[cc]);
    end
    return Es
end

group_numbers (generic function with 1 method)

In [259]:
# #attention: when the dimension is slightly large, converting dense matrix to tensormap is extremely difficult. It's better to define tensormap from Dict
# using TensorKit
# sp=Rep[SU₂](0=>129, 1/2=>196, 1=>185, 3/2=>124, 2=>61, 5/2=>20, 3=>4); #out of memory error for this dimension
# sp=Rep[SU₂](0=>12, 1/2=>19, 1=>15, 3/2=>14, 2=>61, 5/2=>20, 3=>4); # it takes long time
# tt=TensorMap(randn,sp←sp);
# tt_dense=convert(Array,tt);
# tt_new=TensorMap(tt_dense,sp,sp);
function combine_singlespin_sector_bruteforce(E_set,vL_set,vR_set,spin_set)
    E_set_new=copy(euL_set);
    vL_set_new=copy(evL_set);
    vR_set_new=copy(evR_set);
    spin_set_new=copy(SPIN_eig_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0
            single_size=size(convert(Array,vL[1]));
            if length(single_size)==4
                vL_grouped=zeros(Complex{Float64},single_size[1]*length(vL),single_size[2]*single_size[3]*single_size[4]);
                vR_grouped=zeros(Complex{Float64},single_size[2]*single_size[3]*single_size[4],single_size[1]*length(vL));
                E_grouped=zeros(Complex{Float64},single_size[1]*length(vL),single_size[1]*length(vL));
                spin_dim=single_size[1];
                Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
                for cs=1:length(E)
                    vL_grouped[spin_dim*(cs-1)+1:spin_dim*cs,:]=convert(Array,vL[cs]*Unitary);
                    vR_grouped[:,spin_dim*(cs-1)+1:spin_dim*cs]=convert(Array,Unitary'*vR[cs]);
                    E_grouped[spin_dim*(cs-1)+1:spin_dim*cs,spin_dim*(cs-1)+1:spin_dim*cs]=E[cs]*Matrix(I, spin_dim,spin_dim);
                end
                space_single_spin=SU₂Space(spin[1]=>length(E));

                vL_grouped=TensorMap(vL_grouped,space_single_spin←fuse(domain(vL[1])));
                vR_grouped=TensorMap(vR_grouped,fuse(domain(vL[1])) ← space_single_spin);
                E_grouped=TensorMap(E_grouped,space_single_spin ← space_single_spin);

                vL_grouped=vL_grouped*Unitary';
                vR_grouped=Unitary * vR_grouped;

                vL_set_new[cc]=vL_grouped;
                vR_set_new[cc]=vR_grouped;
                E_set_new[cc]=E_grouped;
                spin_set_new[cc]=spin_set[cc][1]
                # display(Base.summarysize(vL_grouped))
                # display(space(vL_grouped))
                # display(Base.summarysize(vL))
            elseif length(single_size)==3
            end
        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end


combine_singlespin_sector_bruteforce (generic function with 1 method)

In [283]:
function combine_singlespin_sector(E_set,vL_set,vR_set,spin_set,is_eig)
    E_set_new=copy(E_set);
    vL_set_new=copy(vL_set);
    vR_set_new=copy(vR_set);
    spin_set_new=copy(spin_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0

            sec=Irrep[SU₂](spin[1]);
            space_single_spin=SU₂Space(spin[1]=>length(E));
            dim_full=size(convert(Dict,vL[1])[:data][string(sec)],2);
            vL_grouped=zeros(Complex{Float64},length(vL),dim_full);
            vR_grouped=zeros(Complex{Float64},dim_full,length(vL));
            E_grouped=zeros(Complex{Float64},length(vL),length(vL));

            for cs=1:length(E)
                vL_grouped[cs,:]=convert(Dict,vL[cs])[:data][string(sec)];
                vR_grouped[:,cs]=convert(Dict,vR[cs])[:data][string(sec)];
                E_grouped[cs,cs]=E[cs];
            end

            if is_eig
                Q=zeros(Complex{Float64},length(E),length(E))
                for ca=1:length(E)
                    for cb=1:length(E)
                        Q[ca,cb]=tr(vL[ca]*vR[cb])
                    end
                end
                #Q=vL_grouped*vR_grouped;#the normalization of this line is inccorect, due to the dimension of spin space
                vL_grouped=pinv(Q)*vL_grouped;
            end

            vL_grouped_rand=TensorMap(randn,space_single_spin←domain(vL[1]));
            vR_grouped_rand=TensorMap(randn,domain(vL[1]) ← space_single_spin);
            E_grouped_rand=TensorMap(randn,space_single_spin ← space_single_spin);

            vL_grouped_dict=convert(Dict,vL_grouped_rand);
            vL_grouped_dict[:data][string(sec)]=vL_grouped;
            vL_grouped=convert(TensorMap,vL_grouped_dict);

            vR_grouped_dict=convert(Dict,vR_grouped_rand);
            vR_grouped_dict[:data][string(sec)]=vR_grouped;
            vR_grouped=convert(TensorMap,vR_grouped_dict);

            E_grouped_dict=convert(Dict,E_grouped_rand);
            E_grouped_dict[:data][string(sec)]=E_grouped;
            E_grouped=convert(TensorMap,E_grouped_dict);

            vL_set_new[cc]=vL_grouped;
            vR_set_new[cc]=vR_grouped;
            E_set_new[cc]=E_grouped;
            spin_set_new[cc]=spin_set_new[cc][1]
            # display(Base.summarysize(vL_grouped))
            # display(space(vL_grouped))
            # display(Base.summarysize(vL))

        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end



combine_singlespin_sector (generic function with 1 method)

In [ ]:
function group_singlespin_sector(group_size,E_set,vL_set,vR_set,spin_set,OO_transform,OO_unitary)
    #the input tensors are combined tensor, where vL and vR correspond to each other after correct transformation

    E_set_new=copy(E_set);
    vL_set_new=copy(vL_set);
    vR_set_new=copy(vR_set);
    spin_set_new=copy(spin_set);
    DTrun_set=copy(E_set);
    
    
    for cc=1:length(E_set)
        E=E_set_new[cc];
        vL=vL_set_new[cc];
        vR=vR_set_new[cc];
        spin=spin_set_new[cc];

        if vL==[]
            DTrun=0;
        else
            DTrun=Int(dim(codomain(vL))/(2*spin[1]+1));#sector size
        end
        
        DTrun_list=[];
        
        #determine grouped structure
        if (0<DTrun)&(DTrun<group_size)
            DTrun_list=[1:DTrun];
        elseif DTrun>=group_size

            for cc=1:Int((DTrun-DTrun%group_size)/group_size)
                DTrun_list=vcat(DTrun_list,[((cc-1)*group_size+1):cc*group_size])
            end
            if DTrun%group_size>0
                DTrun_list=vcat(DTrun_list,[(DTrun-(DTrun%group_size)+1):DTrun]);
            end
        elseif DTrun==0
            continue;
        end
        
        #group tensors according to the above part
        E_new=Vector{Any}(undef, length(DTrun_list));
        vL_new=Vector{Any}(undef, length(DTrun_list));
        vR_new=Vector{Any}(undef, length(DTrun_list));

        sec=Irrep[SU₂](spin[1]);
        
        vL_fullmatrix=convert(Dict,vL)[:data][string(sec)];
        vR_fullmatrix=convert(Dict,vR)[:data][string(sec)];
        E_fullmatrix=convert(Dict,E)[:data][string(sec)];
        dim_full=size(vL_fullmatrix,2);


        for cg=1:length(DTrun_list)

            DTrun_comp=DTrun_list[cg];
            
            space_single_spin=SU₂Space(spin[1]=>length(DTrun_comp));
            
            vL_comp=vL_fullmatrix[DTrun_comp,:];
            vR_comp=vR_fullmatrix[:,DTrun_comp];
            E_comp=E_fullmatrix[DTrun_comp,DTrun_comp];

            vL_comp_rand=TensorMap(randn,space_single_spin←domain(vL));
            vR_comp_rand=TensorMap(randn,domain(vL) ← space_single_spin);
            E_comp_rand=TensorMap(randn,space_single_spin ← space_single_spin);

            vL_comp_dict=convert(Dict,vL_comp_rand);
            vL_comp_dict[:data][string(sec)]=vL_comp;
            vL_comp=convert(TensorMap,vL_comp_dict);

            vR_comp_dict=convert(Dict,vR_comp_rand);
            vR_comp_dict[:data][string(sec)]=vR_comp;
            vR_comp=convert(TensorMap,vR_comp_dict);

            E_comp_dict=convert(Dict,E_comp_rand);
            E_comp_dict[:data][string(sec)]=E_comp;
            E_comp=convert(TensorMap,E_comp_dict);


            if OO_transform
                @tensor vL_comp[:]:=vL_comp[-1,-2,1,-5]*OO_unitary[1,-3,-4];
                vL_comp=permute(vL_comp,(1,),(2,3,4,5,));
                @tensor vR_comp[:]:=vR_comp[-1,1,-4,-5]*OO_unitary'[-2,-3,1];
                vR_comp=permute(vR_comp,(1,2,3,4,),(5,));
            end

            vL_new[cg]=vL_comp;
            vR_new[cg]=vR_comp;
            E_new[cg]=E_comp;


        end
        vL_set_new[cc]=vL_new;
        vR_set_new[cc]=vR_new;
        E_set_new[cc]=E_new;
        spin_set_new[cc]=spin[1];
        DTrun_set[cc]=DTrun_list;
    end
    return E_set_new, vL_set_new, vR_set_new, spin_set_new, DTrun_set
end


In [322]:

function combine_singlespin_sector_unitary(E_set,vL_set,vR_set,spin_set,is_eig) #this should not be used for large bond dimension
    E_set_new=copy(E_set);
    vL_set_new=copy(vL_set);
    vR_set_new=copy(vR_set);
    spin_set_new=copy(spin_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0

            sec=Irrep[SU₂](spin[1]);
            space_single_spin=SU₂Space(spin[1]=>length(E));
            dim_full=dim(fuse(domain(vL[1])),sec);
            vL_grouped=zeros(Complex{Float64},length(vL),dim_full);
            vR_grouped=zeros(Complex{Float64},dim_full,length(vL));
            E_grouped=zeros(Complex{Float64},length(vL),length(vL));

            Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));#this step is not possible if the bond dimension is large.
            for cs=1:length(E)
                vL_grouped[cs,:]=convert(Dict,vL[cs]*Unitary)[:data][string(sec)];
                vR_grouped[:,cs]=convert(Dict,Unitary'*vR[cs])[:data][string(sec)];
                E_grouped[cs,cs]=E[cs];
            end

            if is_eig
                Q=zeros(Complex{Float64},length(E),length(E))
                for ca=1:length(E)
                    for cb=1:length(E)
                        Q[ca,cb]=tr(vL[ca]*vR[cb])
                    end
                end
                #Q=vL_grouped*vR_grouped;#the normalization of this line is inccorect, due to the dimension of spin space
                vL_grouped=pinv(Q)*vL_grouped;
            end

            vL_grouped_rand=TensorMap(randn,space_single_spin←fuse(domain(vL[1])));
            vR_grouped_rand=TensorMap(randn,fuse(domain(vL[1])) ← space_single_spin);
            E_grouped_rand=TensorMap(randn,space_single_spin ← space_single_spin);

            vL_grouped_dict=convert(Dict,vL_grouped_rand);
            vL_grouped_dict[:data][string(sec)]=vL_grouped;
            vL_grouped=convert(TensorMap,vL_grouped_dict);
            vL_grouped=vL_grouped*Unitary';

            vR_grouped_dict=convert(Dict,vR_grouped_rand);
            vR_grouped_dict[:data][string(sec)]=vR_grouped;
            vR_grouped=convert(TensorMap,vR_grouped_dict);
            vR_grouped=Unitary*vR_grouped;

            E_grouped_dict=convert(Dict,E_grouped_rand);
            E_grouped_dict[:data][string(sec)]=E_grouped;
            E_grouped=convert(TensorMap,E_grouped_dict);

            vL_set_new[cc]=vL_grouped;
            vR_set_new[cc]=vR_grouped;
            E_set_new[cc]=E_grouped;
            spin_set_new[cc]=spin_set_new[cc][1]
            # display(Base.summarysize(vL_grouped))
            # display(space(vL_grouped))
            # display(Base.summarysize(vL))

        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end


combine_singlespin_sector_unitary (generic function with 1 method)

In [262]:

function truncate_sectors(n,eu_set,ev_set,ev_set2,spin_set)
    eu_full=eu_set[1]
    dim_full=2*spin_set[1].+1
    for cc=2:length(eu_set)
        eu_full=vcat(eu_full, eu_set[cc])
        dim_full=vcat(dim_full, 2*spin_set[cc].+1)
    end
    dim_full=Int.(round.(dim_full))

    order=sortperm(abs.(eu_full),rev=true);
    eu_full_sorted=eu_full[order];
    dim_full_sorted=dim_full[order];

    total_size=dim_full_sorted[1];
    old_value=eu_full_sorted[1];
    for cc=2:length(eu_full_sorted)
        total_size=total_size+dim_full_sorted[cc];
        new_value=eu_full_sorted[cc];
        if total_size>n
            break;
        end
        old_value=new_value;
    end

    for cc=1:length(eu_set)
        spins=spin_set[cc];
        eu=eu_set[cc];
        ev=ev_set[cc];
        inds=findall(x->abs(x)>=abs(old_value), abs.(eu));
        eu=eu[inds];
        ev=ev[inds];
        spins=spins[inds];
        eu_set[cc]=eu;
        ev_set[cc]=ev;
        spin_set[cc]=spins;
        if ~(ev_set2==[])
            ev2=ev_set2[cc];
            ev2=ev2[inds];
            ev_set2[cc]=ev2;
        end
    end
return eu_set,ev_set,ev_set2,spin_set

end


truncate_sectors (generic function with 1 method)

In [263]:
function FLR_eig(A,O,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    euL_set=Vector{Any}(undef, length(sector_set));
    evL_set=Vector{Any}(undef, length(sector_set));
    euR_set=Vector{Any}(undef, length(sector_set));
    evR_set=Vector{Any}(undef, length(sector_set));
    #println("FLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        #println("spin "*string(spin))

        FLR_eig_L(x)=HV_FL(x,A,O);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*4));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*8, tol=1e-14));
            if minimum(abs.(euL))/maximum(abs.(euL))<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                if  info.converged >= minimum([n_eff,dim(full_space,sec)])
                    @warn "number of values converged is not enough"
                end
            end
        end
        @assert abs.(euL)==sort(abs.(euL), rev=true)
        euL=euL[1:minimum([length(euL),n_eff,dim(full_space,sec)])];
        evL=evL[1:minimum([length(evL),n_eff,dim(full_space,sec)])];
        euL_set[cc]=euL;
        evLL=evL;
        evL=Vector{Any}(undef, length(evLL));
        for cs=1:length(evL)
            evL[cs]=permute(evLL[cs],(1,),(2,3,4,))
        end
        evL_set[cc]=evL;
        SPIN_set[cc]=ones(length(euL))*(dim(sec)-1)/2;
        
        FLR_eig_R(x)=HV_FR(x,A,O);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*4));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*4, tol=1e-14));
            if minimum(abs.(euR))/maximum(abs.(euR))<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                if  info.converged >= minimum([n_eff,dim(full_space,sec)])
                    @warn "number of values converged is not enough"
                end
            end
        end
        @assert abs.(euR)==sort(abs.(euR), rev=true)
        euR=euR[1:minimum([length(euR),n_eff,dim(full_space,sec)])];
        evR=evR[1:minimum([length(evR),n_eff,dim(full_space,sec)])];
        euR_set[cc]=euR;
        evRR=evR;
        evR=Vector{Any}(undef, length(evRR));
        for cs=1:length(evR)
            evR[cs]=permute(evRR[cs],(1,2,3,),(4,))
        end
        evR_set[cc]=evR;

        @assert (norm(abs.(euL)-abs.(euR))/norm(euL))<1e-8
    end
    return euL_set,evL_set,euR_set,evR_set,SPIN_set     
end
        
function HV_FL(vl,A,mpo)
    @tensor vl[:]:=vl[-1,1,3,5]*A'[1,-2,2]*mpo[3,4,-3,2]*A[5,-4,4];
    return vl
end

function HV_FR(vr,A,mpo)
    @tensor vr[:]:=A'[-1,1,2]*mpo[-2,4,3,2]*A[-3,5,4]*vr[1,3,5,-4];
    return vr
end
 
        


HV_FR (generic function with 1 method)

In [264]:
function GLR_eig(A,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    eul_set=Vector{Any}(undef, length(sector_set));
    evl_set=Vector{Any}(undef, length(sector_set));
    eur_set=Vector{Any}(undef, length(sector_set));
    evr_set=Vector{Any}(undef, length(sector_set));
    #println("GLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        #println("spin "*string(spin))
        
        GLR_eig_L(x)=HV_L_tensor(x,A,[]);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*4));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*4, tol=1e-14));
            if minimum(abs.(eul))/maximum(abs.eu(l))<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                if  info.converged >= minimum([n_eff,dim(full_space,sec)])
                    @warn "number of values converged is not enough"
                end
            end
        end
        @assert abs.(eul)==sort(abs.(eul), rev=true)
        
        eul=eul[1:minimum([length(eul),n_eff,dim(full_space,sec)])];
        evl=evl[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eul_set[cc]=eul;
        evll=evl;
        evl=Vector{Any}(undef, length(evll));
        for cs=1:length(evl)
            evl[cs]=permute(evll[cs],(1,),(2,3,))
        end
        evl_set[cc]=evl;
        spin_set[cc]=ones(length(eul))*(dim(sec)-1)/2;

        GLR_eig_R(x)=HV_R_tensor(x,A,[]);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*4));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*8, tol=1e-14));
            if minimum(abs.(eur))/maximum(abs.(eur))<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                if  info.converged >= minimum([n_eff,dim(full_space,sec)])
                    @warn "number of values converged is not enough"
                end
            end
        end
        @assert abs.(eur)==sort(abs.(eur), rev=true)
        eur=eur[1:minimum([length(eur),n_eff,dim(full_space,sec)])];
        evr=evr[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eur_set[cc]=eur;
        evrr=evr;
        evr=Vector{Any}(undef, length(evrr));
        for cs=1:length(evr)
            evr[cs]=permute(evrr[cs],(1,2,),(3,))
        end
        evr_set[cc]=evr;

        @assert (norm(abs.(eul)-abs.(eur))/norm(eul))<1e-8

    end
    return eul_set,evl_set,eur_set,evr_set,spin_set   
end
    
    
    

GLR_eig (generic function with 1 method)

In [265]:
function FLR_svd(A,O,pow,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    S_set=Vector{Any}(undef, length(sector_set));
    U_set=Vector{Any}(undef, length(sector_set));
    Vh_set=Vector{Any}(undef, length(sector_set));
    #display("FLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        #display("spin "*string(spin))

        FLR_svd_R(x)=HV_FR_pow(x,A,O,pow);
        FLR_svd_R_conj(x)=HV_FR_conj_pow(x,A,O,pow);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*4);
            if minimum(S)/maximum(S)<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
            end
        catch e
            println("Number of singular values obtained are not enough, use smaller tol")
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*8, tol=(1e-14));
            if minimum(S)/maximum(S)<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                if  info.converged >= minimum([n_eff,dim(full_space,sec)])
                    @warn "number of values converged is not enough"
                end
            end
        end
        @assert abs.(S)==sort(abs.(S), rev=true)
        S=S[1:minimum([length(S),n_eff,dim(full_space,sec)])];
        U=U[1:minimum([length(U),n_eff,dim(full_space,sec)])];
        V=V[1:minimum([length(V),n_eff,dim(full_space,sec)])];

        UU=U;
        U=Vector{Any}(undef, length(UU));
        for cs=1:length(U)
            U[cs]=permute(UU[cs],(1,2,3,),(4,))
        end

        Vh=Vector{Any}(undef, length(V));
        for cs=1:length(Vh)
            vvr=V[cs];
            Vh[cs]=permute(vvr',(4,),(1,2,3,));
        end
        S_set[cc]=S;
        U_set[cc]=U;
        Vh_set[cc]=Vh;
        SPIN_set[cc]=ones(length(S))*(dim(sec)-1)/2;
        
    end
    return S_set,U_set,Vh_set,SPIN_set     
end
            

function HV_FR_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR(vr,A,mpo);
    end
    return vr
end
function HV_FR_conj(vr,A,mpo)
    @tensor vr[:]:=A[1,-1,2]*mpo'[3,4,-2,2]*A'[5,-3,4]*vr[1,3,5,-4];
    return vr
end
function HV_FR_conj_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR_conj(vr,A,mpo);
    end
    return vr
end
    


HV_FR_conj_pow (generic function with 1 method)

In [266]:
function GLR_svd(A,pow,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    s_set=Vector{Any}(undef, length(sector_set));
    u_set=Vector{Any}(undef, length(sector_set));
    vh_set=Vector{Any}(undef, length(sector_set));
    A_noise=TensorMap(randn, codomain(A),domain(A));
    A_noise=A_noise/norm(A_noise)*norm(A)*1e-9;
    #println("GLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        #println("spin "*string(spin))


        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise*0);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise*0);
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*4);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])    
        catch e
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise);
            println("Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol")
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*8, tol=1e-14);
            if minimum(s)/maximum(s)<1e-7
                display("minimal singular value in this sector is quite small, skip checking number of converged values")
            else
                if  info.converged >= minimum([n_eff,dim(full_space,sec)])
                    @warn "number of values converged is not enough"
                end
            end
        end
        @assert abs.(s)==sort(abs.(s), rev=true)
    
        s=s[1:minimum([length(s),n_eff,dim(full_space,sec)])];
        u=u[1:minimum([length(u),n_eff,dim(full_space,sec)])];
        v=v[1:minimum([length(v),n_eff,dim(full_space,sec)])];

        uu=u;
        u=Vector{Any}(undef, length(u));
        for cs=1:length(u)
            u[cs]=permute(uu[cs],(1,2,),(3,))
        end

        vh=Vector{Any}(undef, length(v));
        for cs=1:length(vh)
            vvr=v[cs];
            vh[cs]=permute(vvr',(3,),(1,2,));
        end
        s_set[cc]=s;
        u_set[cc]=u;
        vh_set[cc]=vh;
        spin_set[cc]=ones(length(s))*(dim(sec)-1)/2;
        
    end
    return s_set,u_set,vh_set,spin_set     
end

function HV_R_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
    end
    return vr
end

function HV_R_conj_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_[1,-1,2]*A'[3,-2,2]*vr[1,3,-3];
    end
    return vr
end



HV_R_conj_pow (generic function with 1 method)

In [267]:

function TransfOp_decom(A,O,total_space,total_sec,pow,Dtrun_init,Dtrun_max,trun_tol,method)
    # euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);

    # eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);

    # S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);

    # s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);


    # euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);

    # eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);

    # S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);

    # s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);


    Dstep=50;
    Dtrun=Dtrun_init;
    if method=="eigenvalue_FLR"
        evL_set=Vector{Any}(undef, length(total_sec));
        euR_set=Vector{Any}(undef, length(total_sec));
        evR_set=Vector{Any}(undef, length(total_sec));
        SPIN_eig_set=Vector{Any}(undef, length(total_sec));
        for cs=1:Int(round((Dtrun_max-Dtrun_init)/Dstep))+1
            _,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(A,O,Dtrun,total_space,total_sec);
            euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
            eu=abs.(group_numbers(euR_set));
            eu_normed=eu/maximum(eu); 
            eu_normed=eu_normed.^pow;
            display("FLR_eig, Dtrun="*string(Dtrun)*", minimal eigenvalue: "*string(minimum(eu_normed)))
            if minimum(eu_normed)<trun_tol
                break;
            else
                if Dtrun>dim(total_space)
                    display("Dtrun exceeds matrix size")
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        
        eu=abs.(group_numbers(euR_set));
        
        for cc=1:length(euR_set)
            euR=euR_set[cc];
            evL=evL_set[cc];
            evR=evR_set[cc];
            SPIN_eig=SPIN_eig_set[cc];
            for cs=length(euR):-1:1
                if ((abs(euR[cs]))/maximum(eu))^pow<trun_tol
                    deleteat!(euR, cs);
                    deleteat!(evR, cs);
                    deleteat!(evL, cs);
                    deleteat!(SPIN_eig, cs);
                end
            end
            euR_set[cc]=euR;
            evL_set[cc]=evL;
            evR_set[cc]=evR;
            SPIN_eig_set[cc]=SPIN_eig;
        end
        
        return euR_set,evL_set,evR_set,SPIN_eig_set

    elseif method=="eigenvalue_GLR"
        evl_set=Vector{Any}(undef, length(total_sec));
        eur_set=Vector{Any}(undef, length(total_sec));
        evr_set=Vector{Any}(undef, length(total_sec));
        spin_eig_set=Vector{Any}(undef, length(total_sec));
        for cs=1:Int(round((Dtrun_max-Dtrun_init)/Dstep))+1
            _,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(A,Dtrun,total_space,total_sec);
            eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
            eu=abs.(group_numbers(eur_set));
            eu_normed=eu/maximum(eu); 
            eu_normed=eu_normed.^pow;
            display("GLR_eig, Dtrun="*string(Dtrun)*", minimal eigenvalue: "*string(minimum(eu_normed)))
            if minimum(eu_normed)<trun_tol
                break;
            else
                if Dtrun>dim(total_space)
                    display("Dtrun exceeds matrix size")
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end

        eu=abs.(group_numbers(eur_set));
        
        for cc=1:length(eur_set)
            eur=eur_set[cc];
            evl=evl_set[cc];
            evr=evr_set[cc];
            spin_eig=spin_eig_set[cc];
            for cs=length(eur):-1:1
                if ((abs(eur[cs]))/maximum(eu))^pow<trun_tol
                    deleteat!(eur, cs);
                    deleteat!(evr, cs);
                    deleteat!(evl, cs);
                    deleteat!(spin_eig, cs);
                end
            end
            eur_set[cc]=eur;
            evl_set[cc]=evl;
            evr_set[cc]=evr;
            spin_eig_set[cc]=spin_eig;
        end
        
        return eur_set,evl_set,evr_set,spin_eig_set

    elseif method=="svd_FLR"
        S_set=Vector{Any}(undef, length(total_sec));
        U_set=Vector{Any}(undef, length(total_sec));
        Vh_set=Vector{Any}(undef, length(total_sec));
        SPIN_svd_set=Vector{Any}(undef, length(total_sec));
        for cs=1:Int(round((Dtrun_max-Dtrun_init)/Dstep))+1
            S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(A,O,pow,Dtrun,total_space,total_sec);
            S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);  
            eu=abs.(group_numbers(S_set));
            eu_normed=eu/maximum(eu); 
            display("FLR_svd, Dtrun="*string(Dtrun)*", minimal eigenvalue: "*string(minimum(eu_normed)))
            if minimum(eu_normed)<trun_tol
                break;
            else
                if Dtrun>dim(total_space)
                    display("Dtrun exceeds matrix size")
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        
        eu=abs.(group_numbers(S_set));
        
        for cc=1:length(S_set)
            S=S_set[cc];
            U=U_set[cc];
            Vh=Vh_set[cc];
            SPIN_svd=SPIN_svd_set[cc];
            for cs=length(S):-1:1
                if ((abs(S[cs]))/maximum(eu))<trun_tol
                    deleteat!(S, cs);
                    deleteat!(U, cs);
                    deleteat!(Vh, cs);
                    deleteat!(SPIN_svd, cs);
                end
            end
            S_set[cc]=S;
            U_set[cc]=U;
            Vh_set[cc]=Vh;
            SPIN_svd_set[cc]=SPIN_svd;
        end
        
        return S_set,U_set,Vh_set,SPIN_svd_set
    elseif method=="svd_GLR"
        s_set=Vector{Any}(undef, length(total_sec));
        u_set=Vector{Any}(undef, length(total_sec));
        vh_set=Vector{Any}(undef, length(total_sec));
        spin_svd_set=Vector{Any}(undef, length(total_sec));
        for cs=1:Int(round((Dtrun_max-Dtrun_init)/Dstep))+1
            s_set,u_set,vh_set,spin_svd_set=GLR_svd(A,pow,Dtrun,total_space,total_sec);
            s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
            eu=abs.(group_numbers(s_set));
            eu_normed=eu/maximum(eu); 
            display("GLR_svd, Dtrun="*string(Dtrun)*", minimal eigenvalue: "*string(minimum(eu_normed)))
            if minimum(eu_normed)<trun_tol
                break;
            else
                if Dtrun>dim(total_space)
                    display("Dtrun exceeds matrix size")
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
    
        eu=abs.(group_numbers(s_set));
        
        for cc=1:length(s_set)
            s=s_set[cc];
            u=u_set[cc];
            vh=vh_set[cc];
            spin_svd=spin_svd_set[cc];
            for cs=length(s):-1:1
                if ((abs(s[cs]))/maximum(eu))<trun_tol
                    deleteat!(s, cs);
                    deleteat!(u, cs);
                    deleteat!(vh, cs);
                    deleteat!(spin_svd, cs);
                end
            end
            s_set[cc]=s;
            u_set[cc]=u;
            vh_set[cc]=vh;
            spin_svd_set[cc]=spin_svd;
        end
        return s_set,u_set,vh_set,spin_svd_set
    end
end
    


TransfOp_decom (generic function with 1 method)

In [298]:
check_complete_decomp=false;
if check_complete_decomp

    euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
    display(size(euL_set))
    eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
    display(size(eul_set))
    S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
    display(size(S_set))
    s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
    display(size(s_set))


    euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
    display(euR_set)
    eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
    display(eur_set)
    S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
    display(S_set)
    s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
    display(s_set)




    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for cc=1:pow-1
                euR=euR*euR0;
            end

            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))




    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for cc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))





    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))



    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

end


In [296]:

Dtrun_init=200;
Dtrun_max=200;
trun_tol=1e-5;

euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
display("SU2 reduced DTrun="*string(length(group_numbers(euR_set))))

# display(euR_set)

    
eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
display("SU2 reduced DTrun="*string(length(group_numbers(eur_set))))
# display(eur_set)


S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
display("SU2 reduced DTrun="*string(length(group_numbers(S_set))))
# display(S_set)


s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
display("SU2 reduced DTrun="*string(length(group_numbers(s_set))))
# display(s_set)


"FLR_eig, Dtrun=200, minimal eigenvalue: 4.146796287759125e-7"

48

"GLR_eig, Dtrun=200, minimal eigenvalue: 5.525824144652258e-12"

22

"minimal singular value in this sector is quite small, skip checking number of converged values"

...

46

"GLR_svd, Dtrun=200, minimal eigenvalue: 1.7974104743332163e-12"

22

Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol


┌ Warning: Invariant subspace of dimension 8 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 201`); setting `howmany = 8`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 21 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number of requested singular values (i.e. `howmany == 201`); setting `howmany = 21`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 13 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 101`); setting `howmany = 13`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 30 (up to requested tolerance `tol = 1.0e-14`), which is smaller than the number

In [297]:

display("check FLR_eig")
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
M0=M;
for cc=1:pow-1
    M=M*M0;
end
M_temp=M*0;
for cc=1:length(euR_set)
    euR=euR_set[cc];
    evR=evR_set[cc];
    evL=evL_set[cc];
    spin=SPIN_eig_set[cc]
    for cs=1:length(euR)
        @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
        M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
    end
end
display(norm(M-M_temp)/norm(M))

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
M_temp=M*0;
for cc=1:length(euR_set)
    euR=euR_set_combined[cc];
    evR=evR_set_combined[cc];
    evL=evL_set_combined[cc];
    spin=SPIN_eig_set_combined[cc]
    if euR==[]
    else
        euR0=euR
        for cc=1:pow-1
            euR=euR*euR0;
        end
        M_temp=M_temp+evR*euR*evL*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))




display("check GLR_eig")
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
M0=M;
for cc=1:pow-1
    M=M*M0;
end
eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

M_temp=M*0;
for cc=1:length(eul_set)
    eur=eur_set_combined[cc];
    evl=evl_set_combined[cc];
    evr=evr_set_combined[cc];
    spin=spin_eig_set_combined[cc]
    if eur==[]
    else
        eur0=eur;
        for cc=1:pow-1
            eur=eur*eur0;
        end
        M_temp=M_temp+evr*eur*evl*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))





display("check FLR_svd")
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
M0=M;
for cc=1:pow-1
    M=M*M0;
end
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set[cc];
    U=U_set[cc];
    Vh=Vh_set[cc];
    spin=SPIN_svd_set[cc]
    for cs=1:length(S)
        M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set_combined[cc];
    U=U_set_combined[cc];
    Vh=Vh_set_combined[cc];
    spin=SPIN_svd_set_combined[cc]
    if S==[]
    else
        M_temp=M_temp+U*S*Vh*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))



display("check GLR_svd")
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
M0=M;
for cc=1:pow-1
    M=M*M0;
end
M_temp=M*0;
for cc=1:length(s_set)
    s=s_set[cc];
    u=u_set[cc];
    vh=vh_set[cc];
    spin=spin_svd_set[cc]
    for cs=1:length(s)
        M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


M_temp=M*0;
for cc=1:length(s_set)
    s=s_set_combined[cc];
    vh=vh_set_combined[cc];
    u=u_set_combined[cc];
    spin=spin_svd_set_combined[cc]
    if s==[]
    else
        M_temp=M_temp+u*s*vh*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))


"check FLR_eig"

0.00011601424799297982

0.00011601424799306828

"check GLR_eig"

1.1540062377815086e-5

...

3.361152293556429e-5

3.361152293556478e-5

"check GLR_svd"

4.586685035797497e-6

4.586685035797349e-6

group_singlespin_sector (generic function with 1 method)

In [463]:

group_size=Int(round((10^8)/(chi*chi*W*W*D)));



1667

In [464]:
check_incomplete_decomp_error=false;
if check_incomplete_decomp_error

    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for ccc=1:pow-1
                euR=euR*euR0;
            end
            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,true,U_fuse_chichi)
    display("group information:")
    display(DTrun_FLR_eig)
    M_temp1=M_temp*0;
    for cc=1:length(euR_set)
        euRs=euR_set_grouped[cc];
        evRs=evR_set_grouped[cc];
        evLs=evL_set_grouped[cc];
        spin=SPIN_eig_set_grouped[cc]
        if euRs==[]
        else
            for cs=1:length(euRs)
                euR0=euRs[cs];
                euR=euR0;
                for ccc=1:pow-1
                    euR=euR*euR0;
                end
                @tensor evR[:]:=U_fuse_chichi[-2,1,2]*evRs[cs][-1,1,2,-3,-4];
                evR=permute(evR,(1,2,3,),(4,));
                @tensor evL[:]:=evLs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                evL=permute(evL,(1,),(2,3,4,));
                M_temp1=M_temp1+evR*euR*evL*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for ccc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))


    eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
    display("group information:")
    display(Dtrun_GLR_eig)
    M_temp1=M_temp*0;
    for cc=1:length(eur_set)
        eurs=eur_set_grouped[cc];
        evrs=evr_set_grouped[cc];
        evls=evl_set_grouped[cc];
        spin=spin_eig_set_grouped[cc]
        if eurs==[]
        else
            for cs=1:length(eurs)
                eur0=eurs[cs];
                eur=eur0;
                for ccc=1:pow-1
                    eur=eur*eur0;
                end

                M_temp1=M_temp1+evrs[cs]*eur*evls[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))




    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))


    S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,true,U_fuse_chichi)
    display("group information:")
    display(DTrun_FLR_svd)
    M_temp1=M_temp*0;
    for cc=1:length(S_set)
        Ss=S_set_grouped[cc];
        Us=U_set_grouped[cc];
        Vhs=Vh_set_grouped[cc];
        spin=SPIN_svd_set_grouped[cc]
        if Ss==[]
        else
            for cs=1:length(Ss)
                S=Ss[cs];
                @tensor U[:]:=U_fuse_chichi[-2,1,2]*Us[cs][-1,1,2,-3,-4];
                U=permute(U,(1,2,3,),(4,));
                @tensor Vh[:]:=Vhs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                Vh=permute(Vh,(1,),(2,3,4,));
                M_temp1=M_temp1+U*S*Vh*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
    display("group information:")
    display(Dtrun_GLR_svd)
    M_temp1=M_temp*0;
    for cc=1:length(s_set)
        ss=s_set_grouped[cc];
        us=u_set_grouped[cc];
        vhs=vh_set_grouped[cc];
        spin=spin_svd_set_grouped[cc]
        if ss==[]
        else
            for cs=1:length(ss)
                s=ss[cs];
                M_temp1=M_temp1+us[cs]*s*vhs[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))
end